### Get TikTok data

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import requests
from datetime import timedelta

In [2]:
today = (dt.datetime.today() + timedelta(days=30)).strftime('%Y-%m-%d')
today

'2022-02-22'

In [3]:
payload={}
headers = {
  'X-API-KEY': 'QT8K1SQ333HurjoXkTVKOchKswsI83cS'
}

In [4]:
count = '30'
secUid = 'MS4wLjABAAAAn982fxeWVi0MRU_Q0Lg0mVJ9AmAZ-GvWgjy9rSGReFPs_xkQSFkKoUnkuWTVhryi'

In [5]:
rng = pd.date_range(start='2021-03-01', end = today, freq='M')
df = pd.DataFrame({ 'date' : rng }, )

In [6]:
df['cursor'] = (pd.to_datetime(df['date']) - dt.datetime(1970,1,1)).dt.total_seconds().round().astype(str).str.replace('.0', '', regex=False) + '000'

In [7]:
cursors = df['cursor'].to_list()

In [8]:
datas = []

for c in cursors:
    url = f"https://api.tikapi.io/public/posts?count={count}&secUid={secUid}&cursor={c}"
    r = requests.request("GET", url, headers=headers, data=payload)
    datas.append(r.json())

In [9]:
len(datas)

11

In [10]:
def simple_dict(tiktok_dict):
    to_return = {}
    to_return['user_name'] = d['author']['uniqueId']
    to_return['video_id'] = d['id']
    to_return['video_desc'] = d['desc']
    to_return['video_time'] = d['createTime']
    to_return['video_duration'] = d['video']['duration']
    to_return['n_shares'] = d['stats']['shareCount']
    to_return['n_likes'] = d['stats']['diggCount']
    to_return['n_comments'] = d['stats']['commentCount']
    to_return['n_plays'] = d['stats']['playCount']
    to_return['video_link'] = 'https://www.tiktok.com/@{}/video/{}?lang=en'.format(to_return['user_name'], to_return['video_id'])
    return to_return

In [11]:
dataframe = []

for data in datas:
    for d in data['itemList']:
        dataframe.append(simple_dict(d))

In [12]:
df = pd.DataFrame(dataframe)

In [13]:
df = df.sort_values('video_time', ascending=False).drop_duplicates(subset='video_id').reset_index(drop=True)

In [14]:
df.head()

,user_name,video_id,video_desc,video_time,video_duration,n_shares,n_likes,n_comments,n_plays,video_link
0,jongraz,7056051376929066287,message received loud and CLEAR 🔮🦴🔮 #nobones #...,1642864985,38,3052,71900,1549,267600,https://www.tiktok.com/@jongraz/video/70560513...
1,jongraz,7055678935107030319,buckle up everybody 🔮🦴🔮,1642778270,57,2737,55200,1206,235900,https://www.tiktok.com/@jongraz/video/70556789...
2,jongraz,7054939278937165102,#greenscreen a bones day FOR SURE! thank you a...,1642606056,58,4452,92900,3314,372200,https://www.tiktok.com/@jongraz/video/70549392...
3,jongraz,7054576003494120751,have the best day everyone! remember the RULES...,1642521474,57,2783,65800,1258,305200,https://www.tiktok.com/@jongraz/video/70545760...
4,jongraz,7053988981276495150,#greenscreen 🚨 new merch DROP 🚨 #merch #ground...,1642384795,54,209,11300,177,128700,https://www.tiktok.com/@jongraz/video/70539889...


In [15]:
len(df)

172

In [16]:
df.to_csv('data/processed/jongraz_videos.csv',index=False)